In [1]:
import pandas as pd

In [15]:
sales_data = pd.read_csv("./data/sales.csv")
sales_data.head()

print(sales_data.state_holiday.unique())

sales_data = pd.get_dummies(sales_data, columns=["state_holiday"], drop_first=True)

['0' 'a' 'c' 'b']


In [16]:
sales_data.head()

,Unnamed: 0,store_ID,day_of_week,date,nb_customers_on_day,open,promotion,school_holiday,sales,state_holiday_a,state_holiday_b,state_holiday_c
0,425390,366,4,2013-04-18,517,1,0,0,4422,False,False,False
1,291687,394,6,2015-04-11,694,1,0,0,8297,False,False,False
2,411278,807,4,2013-08-29,970,1,1,0,9729,False,False,False
3,664714,802,2,2013-05-28,473,1,1,0,6513,False,False,False
4,540835,726,4,2013-10-10,1068,1,1,0,10882,False,False,False
